In [14]:
with open(r"C:\Users\hurua\Downloads\tilesproblem_1326658913086500.txt") as file:
    input_data = file.readlines()

def extract_input_parameters(data):
    landscape = data[2:22]
    landscape = [s.replace("  ", " x ") for s in landscape]
    result = []
    len_line = len(landscape[0].split())

    for j in range(0, len_line, 4):
        x, y = 0, 4
        subresult = []

        for k in range(5):
            subresult = []
            for i in range(x, y):
                try:
                    values = landscape[i].strip().split()[j:j+4]
                    subresult.append(values)
                except:
                    print(x, y)
            result.append(subresult)
            y += 4
            x += 4

    landscape = result
    tiles = data[23:25][1].strip('{|}|\n|').split(', ')
    tiles_dict = {tiles[0].split('=')[0]: tiles[0].split('=')[1],
                  tiles[1].split('=')[0]: tiles[1].split('=')[1],
                  tiles[2].split('=')[0]: tiles[2].split('=')[1]}

    targets_data = data[27:31]
    targets_dict = {}
    for line in targets_data:
        lines = line.strip('\n').split(':')
        targets_dict[lines[0]] = lines[1]

    targets = {}
    i = 1
    for element in targets_dict.values():
        targets[i] = int(element)
        i += 1

    solution_key = data[36::]

    return landscape, tiles_dict, targets, solution_key


landscape, tiles, targets, solution_key = extract_input_parameters(input_data)

def check_target_state(current_state, path):
    diff = {key: targets[key] - current_state[key] for key in targets if key in current_state}

    if (path.count('L_shape') > int(tiles['EL_SHAPE'])) or \
            (path.count('Full_block') > int(tiles['FULL_BLOCK'])) or \
            (path.count('Outer_block') > int(tiles['OUTER_BOUNDARY'])):
        return 'tile_count_not_matching'

    if all(value == 0 for value in diff.values()):
        return 'reached'

    if min(diff.values()) < 0:
        return 'greater'

    return None


def Full_block(var):
    res = {}
    count_1=0
    count_2=0
    count_3=0
    count_4=0
    res[1] = count_1
    res[2] = count_2 
    res[3] = count_3
    res[4] = count_4
    return res

def Outer_block(var):
    res = {}
    count_1=sum(sublist[1:3].count('1') for sublist in var[1:3])
    count_2=sum(sublist[1:3].count('2') for sublist in var[1:3])
    count_3=sum(sublist[1:3].count('3') for sublist in var[1:3])
    count_4=sum(sublist[1:3].count('4') for sublist in var[1:3])
    res[1] = count_1
    res[2] = count_2 
    res[3] = count_3
    res[4] = count_4
    return res

def L_shape(var):
    res = {}
    count_1=sum(sublist[1:].count('1') for sublist in var[1:])
    count_2=sum(sublist[1:].count('2') for sublist in var[1:])
    count_3=sum(sublist[1:].count('3') for sublist in var[1:])
    count_4=sum(sublist[1:].count('4') for sublist in var[1:])
    res[1] = count_1
    res[2] = count_2 
    res[3] = count_3
    res[4] = count_4
    return res

functions = [L_shape, Outer_block, Full_block]
path = []
roadmap = []

def apply_function(funct, area):
    return funct(area)

def csp_solver(current_index, current_sum, path, roadmap, functions, Landscape):
    
    if current_index == len(Landscape):

        if check_target(current_sum,path) == 'reached':            
            return True
        else:
            return False
    else:
        #implementation of MRV
        unassigned_functions = [f for f in functions]
        unassigned_functions = sorted(unassigned_functions, key=lambda f: len([v for v in f(Landscape[current_index]).values() if v > 0]))
        for func in unassigned_functions:

            path.append(func.__name__)
            applied_area = apply_function(func, Landscape[current_index])

            sum_dict = {}
            for key in applied_area:
                if key in current_sum:
                    sum_dict[key] = current_sum[key] + applied_area[key]
            
            if check_target(sum_dict, path) == 'greater':
                path.pop()
                continue
            
            if check_target(sum_dict, path) == 'tile_count_not_matching':
                path.pop()
                continue

            current_sum_total = sum_dict
                      
            result = csp_solver(current_index + 1, current_sum_total, path, roadmap, functions, Landscape)
            if result:
                roadmap.append(f' The index : {current_index}, Area : {Landscape[current_index]}, Applied Shape : {func.__name__}, Applied shape result  : {current_sum_total} \n') 
                return True
            else:
                path.pop()

        return False


result = csp_solver(0, {1:0, 2:0, 3:0, 4:0}, path, roadmap, functions, Landscape)
if result:
    for i in range(0, len(roadmap)):
        try :
            print(roadmap[len(roadmap) - i - 1])
        except:
            print('error', i)
    print('\n')
    print( 'Tiles Used Counts :' , 'L shape count :' , path.count('L_shape'), 'Full block count :' , path.count('Full_block'), 'Outer block count :' , path.count('Outer_block'))
    print('\n')
    print("Path found:", path)
    print('\n')
else:
    print("No path found.")

 The index : 0, Area : [['4', '4', '4', '2'], ['1', '3', '2', '1'], ['4', '3', 'x', '1'], ['1', '1', 'x', '1']], Applied Shape : Full_block, Applied shape result  : {1: 0, 2: 0, 3: 0, 4: 0} 

 The index : 1, Area : [['2', 'x', 'x', '2'], ['x', '3', '1', '1'], ['3', '4', '4', '1'], ['x', '4', '3', '4']], Applied Shape : Full_block, Applied shape result  : {1: 0, 2: 0, 3: 0, 4: 0} 

 The index : 2, Area : [['1', '3', '3', '2'], ['2', '2', 'x', '2'], ['x', '2', '1', '2'], ['2', '2', '1', '1']], Applied Shape : Full_block, Applied shape result  : {1: 0, 2: 0, 3: 0, 4: 0} 

 The index : 3, Area : [['2', '2', 'x', '1'], ['4', '1', '1', '3'], ['x', '4', '4', 'x'], ['3', '1', 'x', '3']], Applied Shape : Full_block, Applied shape result  : {1: 0, 2: 0, 3: 0, 4: 0} 

 The index : 4, Area : [['2', '4', '1', '4'], ['2', '4', '4', 'x'], ['1', '4', '1', '4'], ['4', 'x', 'x', '4']], Applied Shape : L_shape, Applied shape result  : {1: 1, 2: 0, 3: 0, 4: 5} 

 The index : 5, Area : [['x', '1', '4', '4'